# T2.4 Los más buscados de Selenium Europa


<p>Queremos facer un código que garde en formato JSON os datos das persoas máis buscadas en Europa por Europol.  

https://eumostwanted.eu/  

Por cada elemento (expediente), meterse nel e facer captura de pantalla. Gardar nun JSON cada persoa os datos que aparecen na súa ficha. Debes gardar tamén a foto en formato BASE64 dentro dun campo "foto" do JSON.  <p>

## Imports necesarios 

In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import json
import base64
import requests
import numpy as np

## Creación del driver y scrappeo con selenium

In [8]:

# Inicializar el driver
driver = webdriver.Firefox()

URL = "https://eumostwanted.eu/"
driver.get(URL)
time.sleep(5)

# Cerrar el botón de cookies si existe
try:
    cookies = driver.find_element(By.ID, 'sort_close')
    cookies.click()
    time.sleep(2)
except Exception as e:
    print("No se encontró el botón de cookies:", e)

# Obtener los href de los elementos con clase 'wantedItem'
wanted_items = driver.find_elements(By.CLASS_NAME, 'wantedItem')
links = [item.get_attribute("href") for item in wanted_items if item.get_attribute("href")]

lista_datos = []

# Extraer datos de cada enlace
for href in links:
    driver.get(URL + href)
    
    try:
        driver_datos = driver.find_element(By.CLASS_NAME, "wanted_top_right") # Parte de la derecha (foto)
        driver_imagen = driver.find_element(By.CLASS_NAME, "wanted_top_left") # Parte de la izquierza (datos)
        nombre_archivo = re.sub(r'\W+', '_', href) + ".png" # Nombre del archivo de la captura 

        time.sleep(1.5)
        driver.save_screenshot(nombre_archivo)

        
        # Funciones para extraer texto
        def extraer_texto_completo(clase): # Extrer texto de los campos que están dentro de un field__item
            try:
                elemento = driver_datos.find_element(By.CLASS_NAME, clase)
                items = elemento.find_elements(By.CLASS_NAME, "field__item")
                return " ".join([item.text.strip() for item in items if item.text.strip()])
            except:
                return None # Si no tiene contenido el campo ponerlo a null 

        def extraer_texto(clase): # Extraer texto si esta directamente en el div 
            try:
                return driver_datos.find_element(By.CLASS_NAME, clase).text.strip()
            except:
                return None # Si no tiene contenido el campo ponerlo a null 

        def extraer_imagen(clase): # Descargar la imagen y convertirla a base64
            try:
                imagen_elemento = driver_imagen.find_element(By.CLASS_NAME, clase).find_element(By.TAG_NAME, "img")
                respuesta = requests.get(imagen_elemento.get_attribute("src"))
                return base64.b64encode(respuesta.content).decode("utf-8")
            except Exception as e:
                return  None

        # Extraer los datos de los campos con los métodos creados
        datos_persona = {
            "Nombre": extraer_texto("field--name-field-w-first-name"),
            "Alias": extraer_texto_completo("field--node-field-alias"),
            "Pais": extraer_texto("field--name-field-enfast-country"),
            "Crimen": extraer_texto_completo("field--name-field-crime"),
            "Sexo": extraer_texto_completo("field--name-field-gender"),
            "Altura": extraer_texto_completo("field--name-field-approximate-height"),
            "Color_ojos": extraer_texto_completo("field--name-field-eye-colour"),
            "Identificadores": extraer_texto_completo("field--node-field-characteristics"),
            "Fecha_nac": extraer_texto_completo("field--name-field-date-of-birth"),
            "Nacionalidad": extraer_texto_completo("field--name-field-nationality"),
            "Etnia": extraer_texto_completo("field--name-field-ethnic-origin"),
            "Idiomas": extraer_texto_completo("field--node-field-languages"),
            "Estado_caso": extraer_texto_completo("field--name-field-state-of-case"),
            "Fecha_publicacion": extraer_texto_completo("field--name-field-w-last-name"),
            "Imagen": extraer_imagen("field__item")
        }

        lista_datos.append(datos_persona)

    except Exception as e:
        print(f"Error al extraer datos de {href}:", e)


with open('masbuscados.json', 'w', encoding='utf-8') as f:
    json.dump(lista_datos, f, ensure_ascii=False, indent=4)

driver.close()




In [10]:
df = pd.read_json('masbuscados.json')

df.head()

,Nombre,Alias,Pais,Crimen,Sexo,Altura,Color_ojos,Identificadores,Fecha_nac,Nacionalidad,Etnia,Idiomas,Estado_caso,Fecha_publicacion,Imagen
0,"LEIJDEKKERS, Joseph Johannes (Jos)",None,by Netherlands,Illicit trafficking in narcotic drugs and psyc...,Male,182 cm,Brown,None,"July 1, 1991 (33 years)",Dutch,European,None,Sentenced to 24 years of prison,"on May 20, 2022, last modified on February 17,...",/9j/4AAQSkZJRgABAQEAYABgAAD//gA7Q1JFQVRPUjogZ2...
1,"KARATZAIDIS, Manolis",None,by Germany,"Murder, grievous bodily injury",Male,170 cm,Brown,Arm(s) - Tattoo(s) Abstract Design Leg(s) - Ta...,"December 24, 1981 (43 years)",Greek,None,German Greek Russian Turkish,Ongoing investigation,"on February 12, 2025, last modified on Februar...",/9j/4AAQSkZJRgABAQEAYABgAAD//gA7Q1JFQVRPUjogZ2...
2,"KHODOR, Abdullah",None,by Luxembourg,"Kidnapping, illegal restraint and hostage-taki...",Male,181 cm,None,None,"December 30, 1998 (26 years)",German Libyan,European,None,Ongoing investigation,"on March 9, 2022, last modified on February 3,...",iVBORw0KGgoAAAANSUhEUgAAAZAAAAH0CAYAAAAT2nuAAA...
3,"OMAIRAT, Walid",None,by Germany,"Murder, grievous bodily injury",Male,158 cm,Brown,None,"January 1, 1978 (47 years)",Lebanese,None,German Lebanese,Ongoing investigation,"on January 22, 2025, last modified on January ...",/9j/4AAQSkZJRgABAQEAYABgAAD//gA7Q1JFQVRPUjogZ2...
4,"DELATTRE, DOMINIQUE","""la vieille"", ""la vioque""",by France,Organised or armed robbery,Male,175 cm,Brown,Arm(s) - Tattoo(s) Mark Arm(s) - Tattoo(s) Word,"March 27, 1962 (62 years)",French,European,French,Sentenced to 20 years of prison,"on December 4, 2024, last modified on January ...",iVBORw0KGgoAAAANSUhEUgAAAZAAAAH0CAYAAAAT2nuAAA...
